### One Hour Fetching Data

In [ ]:
import aiohttp
import asyncio
import pandas as pd
from datetime import datetime

async def fetch_data_batch(session, url, headers, params, retries=3):
    """
    Fetches data for a batch of cryptocurrencies asynchronously with retries.
    """
    for attempt in range(retries):
        try:
            async with session.get(url, headers=headers, params=params) as response:
                if response.status == 200:
                    return await response.json()
                elif response.status == 429:
                    print("Rate limit hit. Waiting before retrying...")
                    await asyncio.sleep(10)  
                else:
                    print(f"Failed to fetch: HTTP {response.status}")
                    return None
        except Exception as e:
            print(f"Error fetching data: {str(e)}")
            await asyncio.sleep(5)
    return None

async def fetch_crypto_data_async(api_key, symbols):
    """
    Fetches cryptocurrency data for multiple symbols asynchronously.
    """
    base_url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest"
    headers = {
        'X-CMC_PRO_API_KEY': api_key,
        'Accept': 'application/json'
    }
    
    batch_size = 10
    symbol_batches = [symbols[i:i + batch_size] for i in range(0, len(symbols), batch_size)]
    tasks = []
    
    async with aiohttp.ClientSession() as session:
        for batch in symbol_batches:
            params = {
                'symbol': ','.join(batch),
                'convert': 'USD'
            }
            tasks.append(fetch_data_batch(session, base_url, headers, params))
        
        responses = await asyncio.gather(*tasks)
    
    all_data = []
    current_date_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    for response in responses:
        if response:
            try:
                for symbol, coin_data in response['data'].items():
                    quote_data = coin_data['quote']['USD']
                    record = {
                        'datetime': current_date_time,
                        'symbol': symbol,
                        'price': quote_data['price'],
                        'volume_24h': quote_data['volume_24h'],
                        'volume_change_24h': quote_data['volume_change_24h'],
                        'percent_change_1h': quote_data['percent_change_1h'],
                        'percent_change_24h': quote_data['percent_change_24h'],
                        'percent_change_7d': quote_data['percent_change_7d'],
                        'percent_change_30d': quote_data['percent_change_30d'],
                        'percent_change_60d': quote_data['percent_change_60d'],
                        'percent_change_90d': quote_data['percent_change_90d'],
                        'market_cap': quote_data['market_cap'],
                        'market_cap_dominance': quote_data['market_cap_dominance'],
                        'fully_diluted_market_cap': quote_data['fully_diluted_market_cap'],
                        'circulating_supply': coin_data.get('circulating_supply'),
                        'total_supply': coin_data.get('total_supply')
                    }
                    all_data.append(record)
            except KeyError as e:
                print(f"Error processing response: {e}")
    
    return pd.DataFrame(all_data)

async def fetch_one_hour_data(api_key, symbols):
    """
    Fetches cryptocurrency data for the past one hour.
    """
    print("Fetching data for the past hour...")
    df = await fetch_crypto_data_async(api_key, symbols)
    if df.empty:
        print("No data fetched.")
    else:
        print(f"Fetched data for {len(df)} cryptocurrencies.")
    
    return df

async def main():
    API_KEY = "6377a87f-a8ee-4c6f-a62b-f63faefc3e20"
    symbols = ["BTC", "ETH", "BNB", "USDT", "ADA", 
               "SOL", "AVAX", "MATIC", "LINK", "XRP", 
               "DOGEVERSE", "SEAL", "WAI", "SPONGEV2", "FLOKI"]
    one_hour_data = await fetch_one_hour_data(API_KEY, symbols)
    return one_hour_data

import nest_asyncio
nest_asyncio.apply()

one_hour_data = asyncio.run(main())
one_hour_data.head(14)


In [ ]:
one_hour_data.count()

In [ ]:
import aiohttp
import asyncio
import pandas as pd
from datetime import datetime

async def fetch_data_batch(session, url, headers, params, retries=3):
    """
    Fetches data for a batch of cryptocurrencies asynchronously with retries.
    """
    for attempt in range(retries):
        try:
            async with session.get(url, headers=headers, params=params) as response:
                if response.status == 200:
                    return await response.json()
                elif response.status == 429:
                    print("Rate limit hit. Waiting before retrying...")
                    await asyncio.sleep(10)
                else:
                    print(f"Failed to fetch: HTTP {response.status}")
                    return None
        except Exception as e:
            print(f"Error fetching data: {str(e)}")
            await asyncio.sleep(5)
    return None

async def fetch_crypto_data_async(api_key, symbols):
    """
    Fetches cryptocurrency data for multiple symbols asynchronously.
    """
    base_url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest"
    headers = {
        'X-CMC_PRO_API_KEY': api_key,
        'Accept': 'application/json'
    }
    
    batch_size = 10
    symbol_batches = [symbols[i:i + batch_size] for i in range(0, len(symbols), batch_size)]
    tasks = []
    
    async with aiohttp.ClientSession() as session:
        for batch in symbol_batches:
            params = {
                'symbol': ','.join(batch),
                'convert': 'USD'
            }
            tasks.append(fetch_data_batch(session, base_url, headers, params))
        
        responses = await asyncio.gather(*tasks)
    
    all_data = []
    current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    for response in responses:
        if response:
            try:
                for symbol, coin_data in response['data'].items():
                    quote_data = coin_data['quote']['USD']
                    record = {
                        'datetime': current_time,
                        'symbol': symbol,
                        'price': quote_data['price'],
                        'volume_24h': quote_data['volume_24h'],
                        'percent_change_1h': quote_data['percent_change_1h'],
                        'percent_change_24h': quote_data['percent_change_24h'],
                        'market_cap': quote_data['market_cap']
                    }
                    all_data.append(record)
            except KeyError as e:
                print(f"Error processing response: {e}")
    
    return pd.DataFrame(all_data)

async def fetch_per_minute_data(api_key, symbols, duration_minutes=60):
    """
    Fetches cryptocurrency data every minute for the specified duration and aggregates into a DataFrame.
    """
    aggregated_data = []
    hour_name = datetime.now().strftime('%Y-%m-%d %H:00')
    
    for _ in range(duration_minutes):
        print("Fetching data for the current minute...")
        df = await fetch_crypto_data_async(api_key, symbols)
        if not df.empty:
            df['hour'] = hour_name
            aggregated_data.append(df)
        else:
            print("No data fetched for this minute.")
        await asyncio.sleep(60)  
    
    # Combine all minute data into a single DataFrame
    final_df = pd.concat(aggregated_data, ignore_index=True)
    return final_df

async def main():
    API_KEY = "6377a87f-a8ee-4c6f-a62b-f63faefc3e20"
    symbols = ["BTC", "ETH", "BNB", "USDT", "ADA", 
               "SOL", "AVAX", "MATIC", "LINK", "XRP", 
               "DOGEVERSE", "SEAL", "WAI", "SPONGEV2"]
    
    # Fetch data every minute for an hour
    minute_data = await fetch_per_minute_data(API_KEY, symbols, duration_minutes=60)
    return minute_data

import nest_asyncio
nest_asyncio.apply()

# Run the main function
minute_data = asyncio.run(main())
print(minute_data.head())


In [1]:
import aiohttp
import asyncio
import pandas as pd
from datetime import datetime, timedelta

async def fetch_data_batch(session, url, headers, params, retries=3):
    """
    Fetches data for a batch of cryptocurrencies asynchronously with retries.
    
    """
    for attempt in range(retries):
        try:
            async with session.get(url, headers=headers, params=params) as response:
                if response.status == 200:
                    return await response.json()
                elif response.status == 429:
                    print("Rate limit hit. Waiting before retrying...")
                    await asyncio.sleep(10)
                else:
                    print(f"Failed to fetch: HTTP {response.status}")
                    return None
        except Exception as e:
            print(f"Error fetching data: {str(e)}")
            await asyncio.sleep(5)
    return None

async def fetch_crypto_data_async(api_key, symbols):
    """
    Fetches cryptocurrency data for multiple symbols asynchronously.
    """
    base_url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest"
    headers = {
        'X-CMC_PRO_API_KEY': api_key,
        'Accept': 'application/json'
    }
    
    batch_size = 10
    symbol_batches = [symbols[i:i + batch_size] for i in range(0, len(symbols), batch_size)]
    tasks = []
    
    async with aiohttp.ClientSession() as session:
        for batch in symbol_batches:
            params = {
                'symbol': ','.join(batch),
                'convert': 'USD'
            }
            tasks.append(fetch_data_batch(session, base_url, headers, params))
        
        responses = await asyncio.gather(*tasks)
    
    all_data = []
    for response in responses:
        if response:
            try:
                for symbol, coin_data in response['data'].items():
                    quote_data = coin_data['quote']['USD']
                    record = {
                        'datetime': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                        'symbol': symbol,
                        'price': quote_data['price'],
                        'volume_24h': quote_data['volume_24h'],
                        'percent_change_1h': quote_data['percent_change_1h'],
                        'percent_change_24h': quote_data['percent_change_24h'],
                        'market_cap': quote_data['market_cap']
                    }
                    all_data.append(record)
            except KeyError as e:
                print(f"Error processing response: {e}")
    
    return pd.DataFrame(all_data)

async def fetch_last_hour_minutely(api_key, symbols):
    """
    Fetches cryptocurrency data for the last hour, capturing details for every minute.
    """
    aggregated_data = []
    current_time = datetime.now()
    start_time = current_time - timedelta(hours=1)
    
    while current_time > start_time:
        print(f"Fetching data for {current_time.strftime('%Y-%m-%d %H:%M:%S')}...")
        df = await fetch_crypto_data_async(api_key, symbols)
        if not df.empty:
            df['minute'] = current_time.strftime('%Y-%m-%d %H:%M:%S')
            aggregated_data.append(df)
        else:
            print("No data fetched for this minute.")
        
        # Move to the previous minute
        current_time -= timedelta(minutes=1)
        await asyncio.sleep(1)  # Avoid hitting rate limits
    
    # Combine all minute data into a single DataFrame
    final_df = pd.concat(aggregated_data, ignore_index=True)
    final_df.sort_values(by='minute', inplace=True)  # Ensure data is sorted by minute
    return final_df

async def main():
    API_KEY = "6377a87f-a8ee-4c6f-a62b-f63faefc3e20"
    symbols = ["BTC", "ETH", "BNB", "USDT", "ADA", 
               "SOL", "AVAX", "MATIC", "LINK", "XRP", 
               "DOGEVERSE", "SEAL", "WAI", "SPONGEV2"]
    
    # Fetch data for the last hour with minute details
    hourly_data = await fetch_last_hour_minutely(API_KEY, symbols)
    return hourly_data

import nest_asyncio
nest_asyncio.apply()

# Run the main function
hourly_data = asyncio.run(main())
print(hourly_data.head())


Fetching data for 2025-01-08 21:06:51...
Fetching data for 2025-01-08 21:05:51...
Fetching data for 2025-01-08 21:04:51...


KeyboardInterrupt: 

In [13]:
hourly_data.to_csv('hourly_data.csv', index=False)

In [ ]:
hourly_data = pd.read_csv('hourly_data.csv')
hourly_data.head()

In [ ]:
hourly_data["symbol"]=="WAI"

In [ ]:
import pandas as pd

bnb_data = hourly_data[hourly_data["symbol"] == "BTC"]
bnb_data.head()


In [ ]:
bnb_data.describe()